# HMEQ_Data¶
## Predict clients who default on their loan

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

ds = pd.read_csv('../input/hmeq-data/hmeq.csv')
ds

## Many row have Non elements, so drop this all row

In [ ]:
df = ds[ds['VALUE'].isnull()]
df = df[df['JOB'].isnull()]
df = df[df['REASON'].isnull()]
df = df[df['DEROG'].isnull()]
df = df[df['DELINQ'].isnull()]
df = df.reset_index()
df = df['index']
df

In [ ]:
ds = ds.drop(df)
ds = ds.reset_index()
ds = ds.drop(['index'], axis = 1)

ds.info()

In [ ]:
ds.describe()

Data Preprocessing

In [ ]:
ds.corr()['BAD'].sort_values()

Groups MORTDUE

In [ ]:
ds['MORTDUE'].mean()

In [ ]:
plt.plot(ds['MORTDUE'])

In [ ]:
ds['MORTDUE'] = ds['MORTDUE'].fillna(ds['MORTDUE'].mean())

In [ ]:
plt.hist(ds['MORTDUE'])

In [ ]:
ds['MORTDUE'] = np.log(ds['MORTDUE'])

Groups VALUE

In [ ]:
ds['VALUE'].mean()

In [ ]:
plt.plot(ds['VALUE'])

In [ ]:
ds['VALUE'] = ds['VALUE'].fillna(ds['VALUE'].mean())

In [ ]:
plt.hist(ds['VALUE'])

In [ ]:
ds['VALUE'] = np.log(ds['VALUE'])

Groups REASON

In [ ]:
ds['REASON'] = ds['REASON'].fillna('DebtCon')

In [ ]:
ds['REASON'].unique()

In [ ]:
plt.hist(ds['REASON'])

In [ ]:
ds['REASON'] = ds['REASON'].replace(['HomeImp', 'DebtCon'], [0, 1])

Groups JOB

In [ ]:
ds['JOB'].unique()

In [ ]:
ds['JOB'].mode()[0]

In [ ]:
ds['JOB'] = ds['JOB'].fillna(ds['JOB'].mode()[0])

In [ ]:
plt.hist(ds['JOB'])

In [ ]:
ds['JOB'] = ds['JOB'].replace(['Other', 'Sales', 'Office', 'Mgr', 'ProfExe', 'Self'], [0, 1, 2, 3, 4, 5])

Groups YOJ

In [ ]:
ds['YOJ'].mean()

In [ ]:
ds['YOJ'] = ds['YOJ'].fillna(ds['YOJ'].mean())

In [ ]:
plt.plot(ds['YOJ'])

In [ ]:
plt.hist(ds['YOJ'])

In [ ]:
ds['YOJ'] = np.log(ds['YOJ']+1)

Groups DEROG

In [ ]:
ds['DEROG'].std()

In [ ]:
ds['DEROG'] = ds['DEROG'].fillna(ds['DEROG'].std())

In [ ]:
plt.plot(ds['DEROG'])

In [ ]:
plt.hist(ds['DEROG'])

Groups DELINQ

In [ ]:
ds['DELINQ'].std()

In [ ]:
ds['DELINQ'] = ds['DELINQ'].fillna(ds['DELINQ'].std())

In [ ]:
plt.plot(ds['DELINQ'])

In [ ]:
plt.hist(ds['DELINQ'])

Groups CLAGE

In [ ]:
ds['CLAGE']

In [ ]:
ds['CLAGE'].mean()

In [ ]:
ds['CLAGE'] = ds['CLAGE'].fillna(ds['CLAGE'].mean())

In [ ]:
plt.plot(ds['CLAGE'])

In [ ]:
plt.hist(ds['CLAGE'])

Groups NINQ

In [ ]:
ds['NINQ'].std()

In [ ]:
ds['NINQ'] = ds['NINQ'].fillna(ds['NINQ'].std())

In [ ]:
plt.plot(ds['NINQ'])

In [ ]:
plt.hist(ds['NINQ'])

Groups DEBTINC

In [ ]:
ds['DEBTINC'].mean()

In [ ]:
ds['DEBTINC'] = ds['DEBTINC'].fillna(ds['DEBTINC'].mean())

In [ ]:
plt.plot(ds['DEBTINC'])

In [ ]:
plt.hist(ds['DEBTINC'])

In [ ]:
ds.info()

In [ ]:
ds = ds.drop('CLNO', axis = 1)
ds

In [ ]:
y = ds['BAD']
X = ds.drop(['BAD'], axis = 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)

Let's check some models

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()
]

log_cols = ["Classifier", "Accuracy"]
log = pd.DataFrame(columns=log_cols)

acc_dict = {}

for clf in classifiers:
    name = clf.__class__.__name__
    clf.fit(X_train, y_train)
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
        
    if name in acc_dict:
        acc_dict[name] += acc
    else:
        acc_dict[name] = acc

for clf in acc_dict:
    acc_dict[clf] = acc_dict[clf]
    log_entry = pd.DataFrame([[clf, acc_dict[clf]]], columns = log_cols)
    log = log.append(log_entry)

plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')

sns.set_color_codes("muted")
sns.barplot(x = 'Accuracy', y = 'Classifier', data = log, color = "b")

In [ ]:
acc_dict

Best model RandomForestClassifier, take it and use with new parametrs

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

print('score=',rfc.score(X_test, y_test))
y_pred = rfc.predict(X_test)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

score= 0.91